# Preparación, limpieza de datos y Construcción del Dataset a Nivel Seller

**Objetivo:**  
Este notebook toma el dataset a nivel ítem, aplica limpieza (nulos, outliers) y transformaciones acordadas a partir del EDA, y luego construye el dataset agregado a nivel seller que se usará para el clustering.

**Fases (CRISP-DM adaptado):**
- Preparación de datos  
- Ingeniería de atributos  

**Pasos esperados:**

- Limpieza de nulos y reglas básicas
- Manejo de outliers
- Agrupar información por seller.
- Calcular métricas clave (número de ítems, stock, precios, reputación, categorías, logística, etc.).
- Generar un dataset final de sellers listo para análisis y modelado.


In [80]:
import pandas as pd
import numpy as np

ruta = "../datos/crudo/data_por_producto.csv"
df = pd.read_csv(ruta)

print(df.shape)
df.head()

(185250, 14)


,tim_day,seller_nickname,titulo,seller_reputation,stock,logistic_type,condition,is_refurbished,price,regular_price,categoria,url,category_id,category_name
0,2024-08-01,5a39342818,Gel Cerave Limpiador Espumoso Piel Normal A Gr...,green_platinum,16155,FBM,new,False,260.93,355.00,BEAUTY PERSONAL CARE,https://articulo.mercadolibre.com.mx/MLM-17883...,BEAUTY PERSONAL CARE,CUIDADO PERSONAL Y BELLEZA
1,2024-08-01,5a39342818,Kit Garnier Express Aclara Serum +crema +gel C...,green_platinum,6253,FBM,new,False,344.26,NaN,BEAUTY PERSONAL CARE,https://articulo.mercadolibre.com.mx/MLM-31259...,BEAUTY PERSONAL CARE,CUIDADO PERSONAL Y BELLEZA
2,2024-08-01,66d9659692,3 Piezas De Minoxidil 5% Anacastel,green_platinum,10066,FBM,new,False,363.00,660.00,HAIR,https://articulo.mercadolibre.com.mx/MLM-19360...,HAIR,CUIDADO DEL CABELLO
3,2024-08-01,5a39342818,Limpiador Cerave Control Imperfecciones Para P...,green_platinum,19678,FBM,new,False,241.00,410.00,BEAUTY PERSONAL CARE,https://articulo.mercadolibre.com.mx/MLM-18485...,BEAUTY PERSONAL CARE,CUIDADO PERSONAL Y BELLEZA
4,2024-08-01,5a39342818,Labial Líquido Maybelline Super Stay Matte Ink...,green_platinum,4862,FBM,new,False,133.47,152.49,FRAGRANCES & MAKE UP,https://articulo.mercadolibre.com.mx/MLM-15825...,FRAGRANCES & MAKE UP,FRAGANCIAS Y MAQUILLAJE


In [81]:
# Se guarda copia por si se desea una comparación posterior
df_copia = df.copy()

## 1. Limpieza de nulos

In [82]:
# Revisión rapida de nulos para confirmar que todo cuadra
df.isna().mean().sort_values(ascending=False)

regular_price        0.730332
seller_reputation    0.012804
price                0.008184
titulo               0.000000
seller_nickname      0.000000
tim_day              0.000000
logistic_type        0.000000
stock                0.000000
is_refurbished       0.000000
condition            0.000000
categoria            0.000000
url                  0.000000
category_id          0.000000
category_name        0.000000
dtype: float64

### Nulos de seller_reputation

In [83]:
print("Filas antes de eliminar seller_reputation null:", df.shape[0])
df = df[~df['seller_reputation'].isna()].copy()
print("Filas después:", df.shape[0])


Filas antes de eliminar seller_reputation null: 185250
Filas después: 182878


- Los sellers con `seller_reputation` nula fueron eliminados, asumiendo que todos los vendedores deben tener reputación, los nulos corresponden a errores de data.

### Nulos de price y regular_price

In [84]:
# 1) price nulo, regular_price no nulo → imputar price
mask_price_null = df['price'].isna() & df['regular_price'].notna()
print("Filas con price nulo y regular_price no nulo:", mask_price_null.sum())

df.loc[mask_price_null, 'price'] = df.loc[mask_price_null, 'regular_price']

# 2) regular_price nulo, price no nulo → imputar regular_price
mask_reg_null = df['regular_price'].isna() & df['price'].notna()
print("Filas con regular_price nulo y price no nulo:", mask_reg_null.sum())

df.loc[mask_reg_null, 'regular_price'] = df.loc[mask_reg_null, 'price']

# 3) Ambos nulos → eliminar fila
mask_both_null = df['price'].isna() & df['regular_price'].isna()
print("Filas con ambos nulos:", mask_both_null.sum())

df = df[~mask_both_null].copy()


Filas con price nulo y regular_price no nulo: 0
Filas con regular_price nulo y price no nulo: 131413
Filas con ambos nulos: 1509


In [85]:
#Verificación 
df[['price', 'regular_price']].isna().sum()


price            0
regular_price    0
dtype: int64

In [86]:
df[['price', 'regular_price']].describe(include='all')
# Se eliminó el outlier de 4mil millones por no tener seller_reputation

,price,regular_price
count,1.813690e+05,1.813690e+05
mean,3.281464e+03,3.370708e+03
std,1.917927e+05,1.918017e+05
min,2.000000e+00,2.000000e+00
25%,2.690000e+02,2.970000e+02
50%,5.590000e+02,5.982200e+02
75%,1.349000e+03,1.400000e+03
max,7.194236e+07,7.194236e+07


### Registros con price en 0 y stock en 0

Se eliminan los registros con price=0 por ser inválidos en un marketplace, y se conserva stock=0 porque puede representar un producto agotado. Los casos price=0 y stock=0 se eliminan directamente, esto nos puede servir para un enriquecimiento de la data (cliente con productos agotados por ejemplo)

In [87]:
# Eliminar registros con price = 0 (inválidos en marketplace)
mask_price_zero = df['price'] == 0
print("Registros con price = 0:", mask_price_zero.sum())
df = df[~mask_price_zero].copy()

# stock = 0 es válido → NO se elimina

Registros con price = 0: 0


- En una exploración previa de los datos se observaban ambos casos, sin embargo, al hacer la limpieza de nulos esos casos fueron filtrados de una vez. De todas maneras se deja el código utilizado con el fin de mostrar que se tuvo en cuenta en la prueba

In [88]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 181369 entries, 0 to 185249
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   tim_day            181369 non-null  object 
 1   seller_nickname    181369 non-null  object 
 2   titulo             181369 non-null  object 
 3   seller_reputation  181369 non-null  object 
 4   stock              181369 non-null  int64  
 5   logistic_type      181369 non-null  object 
 6   condition          181369 non-null  object 
 7   is_refurbished     181369 non-null  bool   
 8   price              181369 non-null  float64
 9   regular_price      181369 non-null  float64
 10  categoria          181369 non-null  object 
 11  url                181369 non-null  object 
 12  category_id        181369 non-null  object 
 13  category_name      181369 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 19.5+ MB


In [89]:
#Base limpia ;)

## 2. Manejo de outliers

In [90]:
# Función para calculo de iQR*3
def iqr_limite(series, multiplicador=3):
    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1
    inf = Q1 - (multiplicador * IQR)
    sup = Q3 + (multiplicador * IQR)
    return inf, sup


In [91]:
price_inf, price_sup = iqr_limite(df['price'])
stock_inf, stock_sup = iqr_limite(df['stock'])
price_inf, price_sup, stock_inf, stock_sup

(np.float64(-2971.0),
 np.float64(4589.0),
 np.float64(-124.0),
 np.float64(170.0))

In [92]:
df['price'].quantile(0.99)

np.float64(25500.0)

In [93]:
df['stock'].quantile(0.995)

np.float64(9998.0)

In [94]:
mask_price_out = (df['price'] < price_inf) | (df['price'] > price_sup)
mask_stock_out = (df['stock'] < stock_inf) | (df['stock'] > stock_sup)

print("Outliers en price:", mask_price_out.sum())
print("Outliers en stock:", mask_stock_out.sum())


Outliers en price: 14362
Outliers en stock: 18945


In [95]:
# Calcular P99
price_p99 = df['price'].quantile(0.99)
stock_p99 = df['stock'].quantile(0.99)

# Máscaras de outliers
mask_price_high = df['price'] > price_p99
mask_stock_high = df['stock'] > stock_p99

print("Registros con price > P99:", mask_price_high.sum())
print("Registros con stock > P99:", mask_stock_high.sum())

# ---- Validación: ver ejemplos de outliers ----

print("\nTop 5 precios más altos:")
display(df[mask_price_high].sort_values('price', ascending=False).head(5))

print("\nTop 5 stocks más altos:")
display(df[mask_stock_high].sort_values('stock', ascending=False).head(5))

print("\n5 ejemplos aleatorios de outliers (price o stock):")
display(df[mask_price_high | mask_stock_high].sample(5, random_state=42))




Registros con price > P99: 1811
Registros con stock > P99: 1814

Top 5 precios más altos:


,tim_day,seller_nickname,titulo,seller_reputation,stock,logistic_type,condition,is_refurbished,price,regular_price,categoria,url,category_id,category_name
125671,2024-08-01,336fb09897,Best Choice Products Juego De Mesa De Competic...,orange,10050,Otro,new,False,71942361.92,71942361.92,OTHER,https://articulo.mercadolibre.com.mx/MLM-13650...,OTHER,OTROS
146595,2024-08-01,ecb89a19e2,Simulador Avion Moto Vuelo Cinemas Cabinas Bic...,green,10,XD,new,False,24987000.00,24987000.00,TOYS AND GAMES,https://articulo.mercadolibre.com.mx/MLM-64830...,TOYS AND GAMES,JUGUETES Y JUEGOS
17369,2024-08-01,5916f2ce4b,3.0ah/2 Pilas Pistola Inalámbrica Hidrolavador...,green_gold,1,XD,new,False,14999999.00,14999999.00,TOOLS,https://articulo.mercadolibre.com.mx/MLM-20846...,TOOLS,HERRAMIENTAS
102140,2024-08-01,ecb89a19e2,Sistema Lineal Das Event 210a Paq 32 Bafles Ac...,green,10,XD,new,False,11990990.00,11990990.00,AUDIO & GENERAL ELECTRONICS,https://articulo.mercadolibre.com.mx/MLM-74120...,AUDIO & GENERAL ELECTRONICS,AUDIO Y ELECTRÓNICA GENERAL
122785,2024-08-01,a07850e8f1,Billete De 20 Pesos Con Seríe Aa0112333 Serie ...,newbie,1,DS,used,False,10000000.00,10000000.00,ANTIQUES & HOBBIES,https://articulo.mercadolibre.com.mx/MLM-14855...,ANTIQUES & HOBBIES,ANTIGÜEDADES Y PASATIEMPOS



Top 5 stocks más altos:


,tim_day,seller_nickname,titulo,seller_reputation,stock,logistic_type,condition,is_refurbished,price,regular_price,categoria,url,category_id,category_name
183142,2024-08-01,ca26454a78,Back Cover Tapa Retráctil De Aluminio Saveiro ...,yellow,99999,XD,new,False,26499.0,26499.0,ACC CARS & VANS,https://articulo.mercadolibre.com.mx/MLM-32650...,ACC CARS & VANS,ACCESORIOS PARA AUTOS Y CAMIONETAS
184825,2024-08-01,1a69e90893,Toma De Aire Para Cofre Pick Up F-250 2017 A 2019,yellow,99999,Otro,new,False,5098.0,5098.0,ACC CARS & VANS,https://articulo.mercadolibre.com.mx/MLM-15065...,ACC CARS & VANS,ACCESORIOS PARA AUTOS Y CAMIONETAS
184137,2024-08-01,1a69e90893,Dodge Ram 1500 2009-2018 Burrera Cromada Lisas...,yellow,99999,XD,new,False,7099.0,7099.0,ACC CARS & VANS,https://articulo.mercadolibre.com.mx/MLM-13455...,ACC CARS & VANS,ACCESORIOS PARA AUTOS Y CAMIONETAS
68863,2024-08-01,c7be7e219e,Bolsa Coach Klare Crossbody Signature Nueva 10...,yellow,99999,XD,new,False,3999.0,3999.0,APPAREL ACCESSORIES,https://articulo.mercadolibre.com.mx/MLM-20260...,APPAREL ACCESSORIES,ACCESORIOS DE VESTIR
178731,2024-08-01,1a69e90893,Aleron Volado Spoiler Chevrolet Cruze 2017 201...,yellow,99999,Otro,new,False,3098.0,3098.0,ACC CARS & VANS,https://articulo.mercadolibre.com.mx/MLM-15065...,ACC CARS & VANS,ACCESORIOS PARA AUTOS Y CAMIONETAS



5 ejemplos aleatorios de outliers (price o stock):


,tim_day,seller_nickname,titulo,seller_reputation,stock,logistic_type,condition,is_refurbished,price,regular_price,categoria,url,category_id,category_name
94246,2024-08-01,3c919b97d1,"Microscopio Binocular Biológico Profesional, M...",yellow,2,XD,new,False,29604.00,29604.00,HEALTH,https://articulo.mercadolibre.com.mx/MLM-73072...,HEALTH,SALUD
76906,2024-08-01,d11338b4f7,15 Vasos Fiesta Con Tapa Y Popote 24oz,green,24574,XD,new,False,494.00,494.00,HOUSEHOLD ITEMS,https://articulo.mercadolibre.com.mx/MLM-80644...,HOUSEHOLD ITEMS,ARTÍCULOS DEL HOGAR
67785,2024-08-01,ee8e754fdb,Botas Martin Extra Grandes De Cuero Transpirab...,green_silver,5600,XD,new,False,938.85,938.85,FOOTWEAR,https://articulo.mercadolibre.com.mx/MLM-30314...,FOOTWEAR,CALZADO
45508,2024-08-01,04ea882e6c,Amplificador Fender 59' Bassman Ltd 120v 21710...,green_platinum,1,XD,new,False,43200.00,43200.00,MUSICAL INSTRUMENTS,https://articulo.mercadolibre.com.mx/MLM-70473...,MUSICAL INSTRUMENTS,INSTRUMENTOS MUSICALES
98450,2024-08-01,082579482b,Concentrador Americano Nuvo Lite 5 Lts Nidek V...,green_platinum,5,XD,new,False,35149.05,36999.00,HEALTH,https://articulo.mercadolibre.com.mx/MLM-86031...,HEALTH,SALUD


In [96]:
df = df[~mask_price_high & ~mask_stock_high].copy()

In [97]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177747 entries, 4 to 185249
Data columns (total 14 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   tim_day            177747 non-null  object 
 1   seller_nickname    177747 non-null  object 
 2   titulo             177747 non-null  object 
 3   seller_reputation  177747 non-null  object 
 4   stock              177747 non-null  int64  
 5   logistic_type      177747 non-null  object 
 6   condition          177747 non-null  object 
 7   is_refurbished     177747 non-null  bool   
 8   price              177747 non-null  float64
 9   regular_price      177747 non-null  float64
 10  categoria          177747 non-null  object 
 11  url                177747 non-null  object 
 12  category_id        177747 non-null  object 
 13  category_name      177747 non-null  object 
dtypes: bool(1), float64(2), int64(1), object(10)
memory usage: 19.2+ MB


- El método IQR*3 se evaluó inicialmente para detectar outliers, pero dado que las distribuciones de price y stock son altamente asimétricas, este enfoque produjo límites superiores demasiado bajos y marcó como atípicos muchos valores válidos. Para evitar una eliminación excesiva, se optó por usar el percentil 99 (P99), que es más adecuado y permite filtrar únicamente los valores extremadamente inusuales sin afectar la estructura general del dataset
- En este dataset existen valores extremos que pueden ser reales (por ejemplo, simuladores de vuelo con un valor gigante o inventarios inusualmente grandes). Sin embargo, desde el punto de vista analítico, estos casos funcionan igual que outliers estadísticos: son observaciones excepcionales que pueden distorsionar las métricas y afectar la estabilidad del clustering. Por ello, es aceptable que el corte en P99 elimine también algunos valores altos válidos. Esta pérdida puntual no afecta el análisis, ya que el objetivo del modelo es capturar patrones generales del marketplace y no comportamientos extremadamente raros que no son representativos de la población

In [98]:
df['price_log'] = np.log1p(df['price'])
df['stock_log'] = np.log1p(df['stock'])

- Se aplica log1p (ln(x+1) esto porque evitamos que de indefinido al sacar el ln de 0) a price y stock para reducir la asimetría extrema de sus distribuciones. Esta transformación comprime valores muy altos sin perder información, estabiliza la escala y evita que unos pocos casos dominen las métricas en el proceso de clustering

In [99]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 177747 entries, 4 to 185249
Data columns (total 16 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   tim_day            177747 non-null  object 
 1   seller_nickname    177747 non-null  object 
 2   titulo             177747 non-null  object 
 3   seller_reputation  177747 non-null  object 
 4   stock              177747 non-null  int64  
 5   logistic_type      177747 non-null  object 
 6   condition          177747 non-null  object 
 7   is_refurbished     177747 non-null  bool   
 8   price              177747 non-null  float64
 9   regular_price      177747 non-null  float64
 10  categoria          177747 non-null  object 
 11  url                177747 non-null  object 
 12  category_id        177747 non-null  object 
 13  category_name      177747 non-null  object 
 14  price_log          177747 non-null  float64
 15  stock_log          177747 non-null  float64
dtypes: bool

## 3. Construcción de Dataset a nivel Seller

In [100]:
import numpy as np

# Flag: tiene descuento (regular_price > price)
df['tiene_descuento'] = df['regular_price'] > df['price']

# Porcentaje de descuento (0 si no aplica o no hay regular_price > 0)
df['pct_descuento'] = 0.0
mask_desc = df['regular_price'] > 0
df.loc[mask_desc, 'discount_pct'] = (
    (df.loc[mask_desc, 'regular_price'] - df.loc[mask_desc, 'price']) 
    / df.loc[mask_desc, 'regular_price']
)


In [101]:
seller_numeric = df.groupby('seller_nickname').agg(
    num_items=('titulo', 'nunique'),
    categorias_unicas=('category_name', 'nunique'),
    categoria_moda=('category_name', lambda x: x.mode().iloc[0] if not x.mode().empty else None),
    precio_mediano=('price', 'median'),
    precio_promedio=('price', 'mean'),
    stock_total=('stock', 'sum'),
    stock_promedio=('stock', 'mean'),
    precio_log_medio=('price_log', 'mean'),
    stock_log_medio=('stock_log', 'mean'),
    pct_descuento_prom=('pct_descuento', 'mean'),
    pct_con_descuento=('tiene_descuento', 'mean')
).reset_index()


In [102]:
seller_numeric.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43308 entries, 0 to 43307
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   seller_nickname     43308 non-null  object 
 1   num_items           43308 non-null  int64  
 2   categorias_unicas   43308 non-null  int64  
 3   categoria_moda      43308 non-null  object 
 4   precio_mediano      43308 non-null  float64
 5   precio_promedio     43308 non-null  float64
 6   stock_total         43308 non-null  int64  
 7   stock_promedio      43308 non-null  float64
 8   precio_log_medio    43308 non-null  float64
 9   stock_log_medio     43308 non-null  float64
 10  pct_descuento_prom  43308 non-null  float64
 11  pct_con_descuento   43308 non-null  float64
dtypes: float64(7), int64(3), object(2)
memory usage: 4.0+ MB


In [103]:
reputacion = (
    df.groupby('seller_nickname')['seller_reputation']
      .first()
      .reset_index()
)

In [104]:
reputacion['seller_reputation'].value_counts()

seller_reputation
green             12802
green_silver       6340
newbie             6093
green_platinum     5746
green_gold         5317
yellow             2571
red                2040
light_green        1256
orange             1143
Name: count, dtype: int64

In [105]:
condicion = df.groupby('seller_nickname').agg(
    pct_new=('condition', lambda x: (x == 'new').mean()),
    pct_used=('condition', lambda x: (x == 'used').mean()),
    pct_ne=('condition', lambda x: (x == 'not_specified').mean()),
    pct_refurbished=('is_refurbished', 'mean')
).reset_index()


In [106]:
condicion.head()

,seller_nickname,pct_new,pct_used,pct_ne,pct_refurbished
0,000631669c,1.0,0.0,0.0,0.0
1,0007153bca,1.0,0.0,0.0,0.0
2,000bee3c3b,1.0,0.0,0.0,0.0
3,000df2bd02,0.0,1.0,0.0,0.0
4,000e27cea2,1.0,0.0,0.0,0.0


In [107]:
condicion.describe()

,pct_new,pct_used,pct_ne,pct_refurbished
count,43308.000000,43308.000000,43308.000000,43308.000000
mean,0.830610,0.168936,0.000455,0.005607
std,0.371096,0.370705,0.019176,0.070200
min,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000


In [108]:
logistica = (
    df.groupby('seller_nickname')['logistic_type']
      .value_counts(normalize=True)
      .unstack(fill_value=0)
      .reset_index()
)

In [109]:
logistica.head()

logistic_type,seller_nickname,DS,FBM,FLEX,Otro,XD
0,000631669c,0.0,0.0,0.0,1.0,0.0
1,0007153bca,0.0,0.0,0.0,0.0,1.0
2,000bee3c3b,0.0,0.0,0.0,0.0,1.0
3,000df2bd02,0.0,0.0,0.0,0.0,1.0
4,000e27cea2,0.0,0.5,0.0,0.0,0.5


In [110]:
df_seller = (
    seller_numeric
        .merge(reputacion, on='seller_nickname', how='left')
        .merge(condicion, on='seller_nickname', how='left')
        .merge(logistica, on='seller_nickname', how='left')
)

In [111]:
df_seller.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43308 entries, 0 to 43307
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   seller_nickname     43308 non-null  object 
 1   num_items           43308 non-null  int64  
 2   categorias_unicas   43308 non-null  int64  
 3   categoria_moda      43308 non-null  object 
 4   precio_mediano      43308 non-null  float64
 5   precio_promedio     43308 non-null  float64
 6   stock_total         43308 non-null  int64  
 7   stock_promedio      43308 non-null  float64
 8   precio_log_medio    43308 non-null  float64
 9   stock_log_medio     43308 non-null  float64
 10  pct_descuento_prom  43308 non-null  float64
 11  pct_con_descuento   43308 non-null  float64
 12  seller_reputation   43308 non-null  object 
 13  pct_new             43308 non-null  float64
 14  pct_used            43308 non-null  float64
 15  pct_ne              43308 non-null  float64
 16  pct_

In [112]:
df_seller.head()

,seller_nickname,num_items,categorias_unicas,categoria_moda,precio_mediano,precio_promedio,stock_total,stock_promedio,precio_log_medio,stock_log_medio,...,seller_reputation,pct_new,pct_used,pct_ne,pct_refurbished,DS,FBM,FLEX,Otro,XD
0,000631669c,1,1,OTROS,799.0,799.0,10,10.0,6.684612,2.397895,...,newbie,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0007153bca,2,1,OTROS,399.0,399.0,55,27.5,5.991465,3.348517,...,green,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,000bee3c3b,2,1,"LIBROS, MULTIMEDIA Y OTROS",382.5,382.5,0,0.0,5.761745,0.000000,...,newbie,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,000df2bd02,1,1,ACCESORIOS PARA AUTOS Y CAMIONETAS,1550.0,1550.0,5,5.0,7.346655,1.791759,...,green,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,000e27cea2,2,2,ARTÍCULOS DEL HOGAR,457.5,457.5,6,3.0,6.060845,1.354025,...,green_silver,1.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.5


In [113]:
df_seller.isna().mean().sort_values(ascending=False)

seller_nickname       0.0
num_items             0.0
categorias_unicas     0.0
categoria_moda        0.0
precio_mediano        0.0
precio_promedio       0.0
stock_total           0.0
stock_promedio        0.0
precio_log_medio      0.0
stock_log_medio       0.0
pct_descuento_prom    0.0
pct_con_descuento     0.0
seller_reputation     0.0
pct_new               0.0
pct_used              0.0
pct_ne                0.0
pct_refurbished       0.0
DS                    0.0
FBM                   0.0
FLEX                  0.0
Otro                  0.0
XD                    0.0
dtype: float64

In [114]:
# Calculo de diversificación de productos
def entropy(series):
    p = series.value_counts(normalize=True)
    return -(p * np.log(p)).sum()

In [115]:
entropia = (
    df.groupby('seller_nickname')['category_name']
      .apply(entropy)
      .reset_index(name='entropia_categorias')
)

In [116]:
pct_categoria_moda = (
    df.groupby('seller_nickname')['category_name']
      .apply(lambda x: x.value_counts(normalize=True).iloc[0])
      .reset_index(name='pct_categoria_moda')
)

In [117]:
df_seller = (
    df_seller
        .merge(entropia, on='seller_nickname', how='left')
        .merge(pct_categoria_moda, on='seller_nickname', how='left')
)

In [118]:
df_seller.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43308 entries, 0 to 43307
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   seller_nickname      43308 non-null  object 
 1   num_items            43308 non-null  int64  
 2   categorias_unicas    43308 non-null  int64  
 3   categoria_moda       43308 non-null  object 
 4   precio_mediano       43308 non-null  float64
 5   precio_promedio      43308 non-null  float64
 6   stock_total          43308 non-null  int64  
 7   stock_promedio       43308 non-null  float64
 8   precio_log_medio     43308 non-null  float64
 9   stock_log_medio      43308 non-null  float64
 10  pct_descuento_prom   43308 non-null  float64
 11  pct_con_descuento    43308 non-null  float64
 12  seller_reputation    43308 non-null  object 
 13  pct_new              43308 non-null  float64
 14  pct_used             43308 non-null  float64
 15  pct_ne               43308 non-null 

In [119]:
df_seller[['entropia_categorias', 'pct_categoria_moda']].describe()

,entropia_categorias,pct_categoria_moda
count,43308.000000,43308.000000
mean,0.287070,0.856787
std,0.507727,0.240872
min,-0.000000,0.071429
25%,-0.000000,0.714286
50%,-0.000000,1.000000
75%,0.636514,1.000000
max,3.313030,1.000000


- Más del 50% de los sellers vende exclusivamente una sola categoría (entropía = 0).
- El vendedor promedio tiene una fuerte especialización, con ~85% de su catálogo concentrado en su categoría principal.
- Un pequeño grupo de sellers multiproducto alcanza entropías hasta 3.31, reflejando catálogos muy variados.
- Estas dos variables capturan adecuadamente la diferencia entre sellers nicho y sellers diversificados, lo cual será útil en el clustering.

In [120]:
df_seller.to_csv("../datos/procesado/data_seller.csv", index=False)